In [ ]:
import argparse
import os
import math
import random
import json
import numpy as np

from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

from encoder import Encoder
from attention import Attention
from decoder import Decoder
from generator import Generator

from data_loader import SpeechDataset, Padding, ToTensor

In [ ]:
with open('labels_dict.json', 'r') as f:
    labels = json.loads(f.read())
    
id2label = {v: k for k, v in labels.items()}

In [ ]:
SIGNAL_INPUT_SIZE = 1500 
TXT_INPUT_SIZE = 135
OUTPUT_DIM = 33
BATCH_SIZE = 32

audio_conf = {'window': 'hamming',
              'window_size' : 0.02,
              'window_stride' : 0.01,
              'sampling_rate': 16000}

val_dataset = SpeechDataset('../../SpeechRecognition.EN/deepspeech.cv/data/cv-other-dev_manifest.csv', 
                            'labels_dict.json',
                            audio_conf,
                            transform=transforms.Compose([Padding(SIGNAL_INPUT_SIZE, TXT_INPUT_SIZE, 'labels_dict.json')]) 
                              )

In [ ]:
torch.cuda.set_device(1)

In [ ]:
NUM_GRU = 6
FEATURE = 161
ENC_HID_DIM = 256
DEC_HID_DIM = 256 
DEC_EMB_DIM = 256
DROPOUT_RATE = 0.0

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device = torch.device('cpu')

encoder = Encoder(FEATURE, NUM_GRU, ENC_HID_DIM, DEC_HID_DIM, DROPOUT_RATE, device)
attention = Attention(enc_hid_dim=ENC_HID_DIM, dec_hid_dim=DEC_HID_DIM)
decoder = Decoder(OUTPUT_DIM, DEC_EMB_DIM, ENC_HID_DIM, DEC_HID_DIM, DROPOUT_RATE, attention)
model = Generator(encoder, decoder, device).to(device)
model = model.eval()

In [ ]:
signal, txt_ids = torch.from_numpy(val_dataset[2]['signal']), torch.from_numpy(val_dataset[2]['transcript'])
signal = signal.type(torch.FloatTensor).to(device)
signal = signal.permute(1, 0)
txt_ids = txt_ids.type(torch.LongTensor).to(device)

In [ ]:
signal.size(), txt_ids.size()

In [ ]:
model.load_state_dict(torch.load('models/rsr_gan.pt'))
model.eval()

In [ ]:
_, _, output = model(signal.view(1, 1500, 161), txt_ids)

In [ ]:
output.size()

In [ ]:
output.max(2)[1].size()

In [ ]:
out_txt = ''
for l in output.max(2)[1][0].cpu().numpy():
    out_txt += id2label[l] 
    
out_txt

In [ ]:
true_txt = ''
for l in txt_ids[0].cpu().numpy():
    true_txt += id2label[l]
    
true_txt